In [1]:
import pandas as pd
import datetime, pytz, time
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt

import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_indicator import Indicators

C:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
def vumanchustrat(Open,High, Low, Close, period, mfimult, fastema, slowema):
    hcl3 = (High + Low + Close)/3
    esa = vbt.talib('EMA').run(hcl3, period).real.to_numpy()
    de = vbt.talib('EMA').run(np.abs(hcl3 - esa), period).real.to_numpy()
    ci = (hcl3 - esa) / (0.015*de)
    wave1 = vbt.talib('EMA').run(ci, period)
    wave2 = vbt.talib('SMA').run(wave1.real.to_numpy(), period).real.to_numpy()

    cross = wave1.close_crossed_above(wave2).to_numpy()

    wave1 = wave1.real.to_numpy()

    mfi  = (Open - Close) / (High - Low) * mfimult

    ema1 = vbt.talib('EMA').run(Close, fastema).real.to_numpy()
    ema2 = vbt.talib('EMA').run(Close, slowema).real.to_numpy()

    lowl = pd.Series(Low.flatten()).rolling(fastema).min().shift(1).to_numpy()
    highh = pd.Series(High.flatten()).rolling(fastema).max(fastema).shift(1).to_numpy()
    entries = (ema1 >= ema2) & (Close > ema1) & (mfi > 0) & (wave1 < 0 ) & (wave2 < wave1) & cross 
    exits = (Close.flatten() < lowl) | (Close.flatten() > highh)

    return entries, exits


VMCstrat = vbt.IndicatorFactory(
    class_name= 'VuManChu',
    short_name = 'VMC',
    input_names = ["Open", "High", "Low", "Close"] ,
    param_names = ["period", "mfimult", "fastema", "slowema"],
    output_names= ["entries", "exits"]
    ).from_apply_func(vumanchustrat)





In [3]:
params = {
    'backtest': {
        'start': '2017-10-1',
        'end': '2018-1-1',
        'length': {
            'weeks': 4  #durée pendant laquelle on optimise les paramètres
        },
        'forward': {
            'weeks': 2  #durée pendant laquelle on joue les paramètres optimisés
        },
        
    },
    'strat':{
        'download_params': {
            'symbols': 'BTCUSDT',
            'interval': '1h',
            'missing_index': 'drop'            
        },
        'get_params': {
            'column': ['Open', 'High', 'Low', 'Close']
        },
        'run_params': { 
            'period': {'start': 10, 'stop': 20, 'step': 1},
            'mfimult': {'start': 1, 'stop': 5, 'step': 0.2},
            'fastema': {'start': 30, 'stop': 70, 'step': 10},
            'slowema': {'start': 100, 'stop': 400, 'step': 50},
            'param_product': 'True'
        }, 
    },
    'portfolio': {'freq': 'H' ,
                  'fees' : 0.1/100,
                  'size_type': 'Percent',
                  'size': .1} #proportion du portefeuille à investir dans chaque trade
}



In [4]:
indics = Indicators()
test = ForwardTest(indics.VMCstrat, params)
test.backtest(init_cash=1000)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-10-29 00:00:00+00:00 and 2017-11-12 00:00:00+00:00 finished at 2023-07-18 20:30:32.060534
Cash: 1000.0


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-12-10 00:00:00+00:00 and 2017-12-24 00:00:00+00:00 finished at 2023-07-18 20:36:27.395256
Cash: 1000.0


0it [00:00, ?it/s]

1000.0

In [5]:
pf = test.aggregate_results()
pf.stats()

Start                         2017-10-29 00:00:00+00:00
End                           2017-12-23 23:00:00+00:00
Period                                 56 days 00:00:00
Start Value                                       100.0
End Value                                         100.0
Total Return [%]                                    0.0
Benchmark Return [%]                          133.38779
Max Gross Exposure [%]                              0.0
Total Fees Paid                                     0.0
Max Drawdown [%]                                    NaN
Max Drawdown Duration                               NaT
Total Trades                                          0
Total Closed Trades                                   0
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 